In [2]:
import getpass
import os
# QhdyPvS28vnpVY94g5YKmm58PgEhUCYb
os.environ["MISTRAL_API_KEY"] = getpass.getpass()

In [3]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
#from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


#### Load the PDF

In [4]:
from langchain.document_loaders import PyPDFLoader

pdf_file_path = "ArtificialIntelligenceAct.pdf"

loader = PyPDFLoader(pdf_file_path)

# Load and process the document
docs = loader.load()

#### Create the Chunks, generate the embeddings and store them
The model we used for generating the embeddings was the ``all-MiniLM-L6-v2`` of Hugging Face.

Then we stored those embeddings in a vectorial data base (Chroma).


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))

C:\Users\perei\AppData\Local\Temp\ipykernel_17048\4226516271.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))
c:\Users\perei\miniconda3\envs\chatbot\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(
                search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.8}
            )
            
prompt = hub.pull("rlm/rag-prompt")

"""Prompt template:
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

    Question: {question} 

    Context: {context} 

    Answer:
"""

print(retriever.get_relevant_documents("Hi"))

c:\Users\perei\miniconda3\envs\chatbot\Lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
C:\Users\perei\AppData\Local\Temp\ipykernel_17048\2823870735.py:19: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(retriever.get_relevant_documents("Hi"))
c:\Users\perei\miniconda3\envs\chatbot\Lib\site-packages\langchain_core\vectorstores\base.py:1083: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'page': 329, 'source': 'ArtificialIntelligenceAct.pdf'}, page_content='The Board may establish other standing or temporary sub-groups as appropriate for the \npurpose of examining specific issues. Where appropriate, representatives of the advisory \nforum referred to in Article 67 may be invited to such sub-groups or to specific meetings \n

[]


In [7]:
print(hub.pull("langchain-ai/rag-context-recall"))

c:\Users\perei\miniconda3\envs\chatbot\Lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['documents', 'student_answer'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'langchain-ai', 'lc_hub_repo': 'rag-context-recall', 'lc_hub_commit_hash': '8b6ffd487ad2a66c0fc129ee1e3d4854336c7fd736405af0b9b1a0ffb63d51e8'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a teacher grading a quiz. \n\nYou will be given a STUDENT ANSWER and a set of FACTS. \n\nHere is the grade criteria to follow:\n(1) Look at each sentence in the STUDENT ANSWER.\n(2) Determine whether the sentence can be attributed to the FACTS.\n(3) Assign a score of 1 if the sentence can be attributed to the FACTS \n(4) Assign a score of 0 if the sentence can NOT be attributed to the FACTS \n\nScore:\nReturn the fraction: The number of sentences that can be attributed to the FACTS and got a score of 1 divided by the total number of sentences.\n\nExplain your reasoning in a step-by-step manner to ensure 

#### Modelo : MistralAI através da Langchain

In [11]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="open-mistral-7b", api_key=os.environ["MISTRAL_API_KEY"])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Pipeline de RAG
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#rag_chain.invoke("What are the main values enshrined in the AI Act, according to Union law?")
#rag_chain.invoke("What are the primary objectives of the Artificial Intelligence Act?")
user_prompt = "How does the AI Act aim to regulate high-risk AI systems?"
rag_chain.invoke(user_prompt)

No relevant docs were retrieved using the relevance score threshold 0.8


'The AI Act aims to regulate high-risk AI systems by establishing a risk-based framework. This includes mandatory risk assessments, transparency requirements, and accountability mechanisms for developers and users. Additionally, the Act proposes a prohibition on certain uses of high-risk AI systems, such as those that pose a significant threat to safety or fundamental rights.'

In [18]:
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

# Construir o grafo adicionando nós
graph_builder.add_node("chatbot", chatbot)

# Conectar o início ao nó do chatbot
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "1"}}

while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    for event in graph.stream({"messages": [("user", user_input)]}, config, stream_mode="values"):
        last_message = event["messages"][-1]  # Acessa a última mensagem

        # Verifica o tipo de mensagem e imprime o conteúdo
        if hasattr(last_message, 'content'):
            print(last_message.content)  # Imprime o conteúdo
        else:
            print("Mensagem não possui conteúdo.")
            
        print("--")

QhdyPvS28vnpVY94g5YKmm58PgEhUCYb
--
It seems like you've provided a hexadecimal string. To provide a meaningful response, I would need more context. Is this a password, a code, or some other kind of data? If it's a password, I should clarify that I can't help you decrypt or crack it as I'm designed to promote ethical and safe use of technology. If it's a code, please specify its purpose or format so I can help you understand or decode it.
--
How does the AI Act aim to regulate high-risk AI systems?
--
The Artificial Intelligence Act, proposed by the European Commission, aims to regulate high-risk AI systems in the European Union. The proposed legislation includes the following key elements:

1. Risk classification: The act categorizes AI systems based on their level of risk, from low to high. High-risk AI systems include those that can cause physical harm to humans or the environment, infringe upon fundamental rights, or have significant societal impacts.
2. Mandatory requirements: Hig

In [ ]:
"""from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph


# Graph
workflow = StateGraph(state_schema=MessagesState)
memory = MemorySaver()
app = workflow.compile(checkpointer=memory) 

# Verificar se o nó "model" já foi adicionado
if "model" not in workflow.nodes:
    workflow.add_node("model")
    # Conectar o início ao nó "model"
    workflow.add_edge(START,"model")

if 'messages' not in locals():
    messages = []

# Função para interagir com o chatbot
def ask_chatbot(user_prompt):
    global messages

    # Atualizar o workflow com a mensagem do usuário
    workflow.add_edge("model", HumanMessage(content=user_prompt))

    # Send the prompt to the LLM
    response = rag_chain.invoke(user_prompt, config={"configuration": {"thread_id": "1"}})

    # Armazenar mensagens
    messages.append({'role': 'user', 'content': user_prompt})
    messages.append({'role': 'assistant', 'content': response})

    # Atualizar o workflow com a resposta do assistente
    workflow.add_edge("model", AIMessage(content=response))

    return response

user_prompt = "How does the AI Act aim to regulate high-risk AI systems?"
response = ask_chatbot(user_prompt)
print("Assistant:", response)"""